In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


#### Document Loader

In [2]:
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
loader=UnstructuredURLLoader(urls=urls)
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle Rating\n\n5 stars, 111 reviews\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n\nApartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring

#### Splitting text in chunks

In [3]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000)
docs=text_splitter.split_documents(docs)
len(docs)

11

In [4]:
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle Rating\n\n5 stars, 111 reviews\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n\nApartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring 

#### Embeddings using Ollama

In [6]:

# embedding_model = OllamaEmbeddings(model="all-minilm")
embedding_model = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model)
vectorstore

#### Querying the Chroma DB

In [7]:
# Retrieve relevant documents
retriever = vectorstore.as_retriever(search_type='similarity',search_kwargs={"k": 3})
query = "OUR SERVICE WE OFFER QUICK & POWERFUL LOGISTICS SOLUTION"
# retrieved_docs = retriever.get_relevant_documents(query)
retrieved_docs = retriever.invoke(query)


# Print retrieved documents
print(retrieved_docs[0].page_content)

OUR SERVICE

WE OFFER QUICK & POWERFUL LOGISTICS SOLUTION

FURNITURE REMOVALS:

We are the furniture removals specialists in Victoria. Our Teams are experienced and comes with Furniture moving equipments like- Trolleys. Blankets, Loading Ramps.

INSURANCE:

Victoria On Move Pvt Ltd. look after your goods arrive safely at your destination. We have Transit and Public liability insurance. Transit insurance covers loading and unloading from vehicle , and Road accident cover.

1 BED HOME:

1 Bed, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine, Some Boxes.

2 BED HOME:

2 Beds, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes, Loose stuff.

Big HOME:

4-5 Bed Room, Double Lounge, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes,Furniture, Loose stuff. Get a Quote

Interstate Removalists:

Seamless Relocation Across State Lines

Melbourne to Sydney Removalists

Book Now


In [8]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama3.2:1b")
llm

C:\Users\Wasib\AppData\Local\Temp\ipykernel_15268\2590928375.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama3.2:1b")


Ollama(model='llama3.2:1b')

#### Design ChatPrompt Template

In [9]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. If you don't know the answer, say that you don't know.
Use three sentences maximum and keep the answer concise. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )

#### Chain Introduction


In [10]:
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [11]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001A18D8228D0>, search_kwargs={})

In [12]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [13]:
response = retrieval_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])

They provide furniture removals, insurance solutions, and various moving services such as apartment moving, villa moving, house moving, office moving, furniture moving, packing and unpacking services, customized moving plans, professional team, local movers, and interstate removalists.
